## The DSL backend

As mentioned in the introduction tutorial, backends are responsible for keeping track of any tensors created/removed during the creation or contraction of tensor networks. The DSL backend does this by writing tensor data, and commands describing the manipulation of it, to disk instead of saving the data in memory and acting on it interactively.

With a DSL backend, when a tensor network is created with PicoQuant, all of the associated tensor data will be written to a HDF5 file. When we call functions to contract or manipulate tensors contained in a tensor network, instead of acting on the tensor data in the HDF5 file directly, the DSL backend will use a domain specfic language to write commands describing the contraction/manipulation. These commands are written to a DSL script which is created when the DSL backend is created. Therefore, for example, contracting a tensor network representing a quantum circuit with a DSL backend will not produce the output of the circuit immediately. To get the output, we must also call an execute_dsl_file function which will read the DSL commands in the DSL script and then perform the operations described there to produce the output.

One advantage of writing a DSL script over operating on tensor data interactively is that it allows for a preprocessing stage, before any tensor operations are carried out with the tensor data, where an efficient execution of the DSL commands can be determined.

Before illustrating the behaviour of PicoQuant functions induced by the DSL backend, we give a brief description of the domain specfic language.

## The Domain Specific Language

The domain specific language consists of the following commands:
1. tensor - Command to load a tensor from the HDF5 file referenced by the DSL backend. Input arguments are: 
    1. A name for a variable to bind to the tensor data when it is loaded into memory 
    2. The name under which the tensor data is saved in the HDF5 file.
    
    
2. del - Command to delete a tensor. Input arguments are:
    1. The vaiable name of a tensor in memory to delete
    
    
3. save - Command to save a tensor to a given output file. Input arguments are:
    1. The tensor variable name, 
    2. The output HDF5 file name,
    3. The name to save the tensor under within the HDF5 file.


4. ncon - Command to contract two tensors $A$ and $B$ together and produce a new tensor $C$. Input arguments are 
    1. A name for the new tensor $C$, 
    2. The name for the first tensor $A$ 
    3. The ncon indices identifying which indices of $A$ to contract, 
    4. The name of the second tensor $B$,
    5. The ncon indices for the second tensor $B$.


5. reshape - Command to reshape a tensor. Input arguments are:
    1. The name of a tensor to reshape,
    2. An array of integers specifying the desired shape of the tensor.
    
    
6. decompose - Command to decompose a tensor using SVD. Input arguments are:
    1. The name of the tensor to decompose,
    2. An array of integers iddentifying the indices of the tensor to be considered the left indices during SVD.
    3. An array of integers iddentifying the right indices of the tensor during SVD.
    4. A real, positive number giving the threshold under which singular values will be discarded.

## Using the DSL Backend

In [ ]:
using PicoQuant

To initialise a DSL backend and set it as the current backend used by PicoQuant we call the following constructor. The input arguments are the names of the DSL script, tensor date file and output date file respectively. (Note, if no output file is given, results will be written to the tensor data file by default.)

In [ ]:
# Initialise a DSL backend.
DSLBackend("dsl_file.tl", "tensor_file.h5", "output_file.h5")

At this point, there should be a DSL backend set as the current backend and there should be an empty DSL script in the current directory. We now create a TensorNetworkCircuit instance, add a few quantum gates to it and then contract it. This will populate the DSL script with commands describing how to contract the tensor data.

In [ ]:
# Create a TensorNetworkCircuit and gate data.
tn = TensorNetworkCircuit(3)
hadamard_data = gate_tensor(:H)
CNOT_data = gate_tensor(:CX)

# The DSL backend will be used by these function to write the given gate data to tensor_file.h5 and 
# write 'tensor' commands to the DSL script.
add_gate!(tn, hadamard_data, [1])
add_gate!(tn, CNOT_data, [2, 1])
add_gate!(tn, CNOT_data, [3, 2])
add_input!(tn, "000")

# This function will use the DSL backend to write ncon commands to the DSL script.
full_wavefunction_contraction!(tn, "vector")

We can now view the contents of the DSL script to see the commands generated by the DSL backend.

In [ ]:
;cat dsl_file.tl

To get the output of the circuit we created, we call the function execute_dsl_file and pass it the DSL script and the tensor data HDF5 file. This will save the output of the circuit under the name "result" in the output file given to the DSL backend when it was created.

In [ ]:
execute_dsl_file("dsl_file.tl", "tensor_file.h5")

We can read the result of the above contraction as follows:

In [ ]:
output = load_tensor_data(backend, :result)